In [ ]:
import fceulib
import ppu_dump
import sys
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2

from fceu_help import pointer_to_numpy

In [ ]:
#rom = "mario.nes"

#movie = 'happylee4-smb-warpless.fm2'

#movie = "mario.fm2"
# start_t = 373
# rom = "metroid.nes"
#movie = "metroid.fm2"
# start_t = 300
# movie ='lordtom-metroid-100.fm2'
#movie = "metroid-long.fm2"

#rom = 'zelda.nes'
#movie = 'baxter,jprofit22-legendofzelda.fm2'
#movie = 'zelda.fm2'

#rom = 'zelda.nes'
#movie = 'zelda_dungeon1.fm2'
#movie = 'zelda.fm2'
#start_t = 300

# rom = "ff.nes"
# movie = 'theaxemanv4-finalfantasy.fm2'
# start_t = 10

rom = 'dw.nes'
movie = 'dw-menus.fm2'
start_t = 10

# rom = "smb2u.nes"
# movie = "smb2u.fm2"
# start_t = 700
emu = fceulib.runGame(rom)
inputs1 = fceulib.readInputs(movie)
inputs2 = fceulib.readInputs2(movie)

for i,i2 in zip(inputs1[:start_t],inputs2[:start_t]):
    emu.stepFull(i, i2)
    
 
def convert_image(img_buffer):
    # TODO: without allocations/reshape?
    screen = pointer_to_numpy(img_buffer)
    return screen.reshape([256, 256, 4])[:240, :, :3].astype(np.uint8)
img_buffer = fceulib.VectorBytes()
np_image = np.zeros(shape=(240, 256, 1), dtype=np.uint8)
emu.imageInto(img_buffer)
np_image = convert_image(img_buffer)
plt.imshow(np_image)
plt.show()

print len(inputs1)

    

In [ ]:
%%time

reload(ppu_dump)

end = -1
#METROID
scroll_area= (0,0,32,30-0)

#ZELDA
#scroll_area= (0,8,32,30-8)

#MARIO, ZELDA2
#scroll_area = (0, 4, 32, 30-4)

ep_data = ppu_dump.ppu_output(emu,
                              inputs1[start_t:end],
                              inputs2=inputs2[start_t:end],
                              bg_data=True,
                              scrolling=True,
                              sprite_data=True,
                              colorized_tiles=False,
                              display=False,
                              test_control=True,
                              peekevery=2,
                            scroll_area= scroll_area )

Let's find overlays on screens.  We can do that starting from the opening menus, so let's give it a shot.  We'll start by looking for rectangles.  That seems helpful.

In [ ]:
emu2 = fceulib.runGame(rom)
inputs1a = fceulib.readInputs(movie)
inputs2a = fceulib.readInputs2(movie)

start_t2 = 170
for i,i2 in zip(inputs1a[:start_t2],inputs2a[:start_t2]):
    emu2.stepFull(i, i2)

emu2.imageInto(img_buffer)
img = convert_image(img_buffer)
plt.imshow(img)
plt.show()

In [ ]:
def find_boxes(img):
    img2 = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    slen = 8
    sopp = 2

    img2 = cv2.GaussianBlur(img2, ksize=(5, 5), sigmaX=0)

    img3h = np.copy(img2)
    hlines = cv2.getStructuringElement(cv2.MORPH_RECT, (slen, sopp))
    img3h = cv2.erode(img3h, hlines, iterations=1)
    img3h = cv2.dilate(img3h, hlines, iterations=1)

    img3v = np.copy(img2)
    vlines = cv2.getStructuringElement(cv2.MORPH_RECT, (sopp, slen))
    img3v = cv2.erode(img3v, vlines, iterations=1)
    img3v = cv2.dilate(img3v, vlines, iterations=1)

    thresh = img3h + img3v

    plt.imshow(thresh)
    plt.show()

    cntrs, _ = cv2.findContours(
        thresh.copy()[8:-8, 8:-8], cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    # loop over the contours
    img2 = np.copy(img)
    seen = []
    for c in cntrs:
        # compute the center of the contour, then detect the name of the
        # shape using only the contour
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.04 * peri, True)
        if len(approx) != 4:
            continue
        if peri < 196:
            continue
        box = cv2.boundingRect(approx + [8, 8])
        area = box[2] * box[3]
        if area > 28 * 8 * 26 * 8:
            continue
        duplicate = False
        M = cv2.moments(approx)
        cX = int((M["m10"] / M["m00"]))
        cY = int((M["m01"] / M["m00"]))
        for M2, _, _ in seen:
            cX2 = int((M2["m10"] / M2["m00"]))
            cY2 = int((M2["m01"] / M2["m00"]))
            ox = cX - cX2
            oy = cY - cY2
            ratio = M["m00"] / M2["m00"]
            print "R", np.abs(1 - ratio), "O", ox, oy
            if np.sqrt(ox * ox + oy * oy) < 32 and np.abs(1 - ratio) < 1:
                print "skip"
                duplicate = True
                break
        if duplicate:
            continue
        seen.append((M, box, approx))
        # multiply the contour (x, y)-coordinates by the resize ratio,
        # then draw the contours and the name of the shape on the image
        cv2.rectangle(img2, box[0:2], (box[0] + box[2],
                                       box[1] + box[3]), (0, 255, 0), 2)
        # show the output image
    print len(seen)
    plt.imshow(img2)
    plt.show()
    return seen


find_boxes(img)

In [ ]:
emu2 = fceulib.runGame(rom)
inputs1a = fceulib.readInputs(movie)
inputs2a = fceulib.readInputs2(movie)

end_t2 = 2600
skip = 30
live_threshold = 30
visible_boxes = {}
live_boxes = []
for t, (i, i2) in enumerate(zip(inputs1a[:end_t2], inputs2a[:end_t2])):
    emu2.stepFull(i, i2)
    if t % skip == 0:
        print "T:", t
        emu2.imageInto(img_buffer)
        img = convert_image(img_buffer)
        here_boxes = find_boxes(img)
        # first, update visibility stats --- remove anything from visible that is not present
        # and increment the count of everything that is present (by frameskip amount).

        hkeys = set(map(lambda (_m, b, _c): (
            b[0] / 16, b[1] / 16, b[2] / 16, b[3] / 16), here_boxes))
        for lk in visible_boxes.keys():
            if lk not in hkeys:
                del visible_boxes[lk]
        for hm, hb, hc in here_boxes:
            hk = (hb[0] / 16, hb[1] / 16, hb[2] / 16, hb[3] / 16)

            oldlive = 0
            if hk in visible_boxes:
                visc, boxdata = visible_boxes[hk]
                oldlive = visc
                visible_boxes[hk] = visc + skip, boxdata
                print "old box now", visc + skip
            else:
                visible_boxes[hk] = skip, (hm, hb, hc)
                print "new box now", skip

            # if something is visible but not in live, add it if it crossed the liveness threshold
            if visible_boxes[hk][0] >= live_threshold and oldlive < live_threshold:
                live_boxes.append((hm, hb, hc))
                print "now live"

        lkeys = set(map(lambda (_m, b, _c): (
            b[0] / 16, b[1] / 16, b[2] / 16, b[3] / 16), live_boxes))
        # then, if something is live but not visible, remove it if it isn't overlapped by anything later in live
        toremove = set()
        for li, (lm, lb, lc) in enumerate(reversed(live_boxes)):
            lk = (lb[0] / 16, lb[1] / 16, lb[2] / 16, lb[3] / 16)
            if lk not in visible_boxes:
                l10x = lb[0]
                l10y = lb[1]
                l11x = lb[0] + lb[2]
                l11y = lb[1] + lb[3]
                overlapped = False
                li_f = len(live_boxes) - li - 1
                for lm2, lb2, lc2 in live_boxes[li_f + 1:]:
                    if lb2 in toremove:
                        continue
                    l20x = lb2[0]
                    l20y = lb2[1]
                    l21x = lb2[0] + lb2[2]
                    l21y = lb2[1] + lb2[3]
                    if (l10x <= l21x and l20x <= l11x) and (l10y <= l21y and l20y <= l11y):
                        overlapped = True
                        break
                if not overlapped:
                    print "now dead"
                    toremove.add(lb)
        for b in toremove:
            for li, (_lm, lb, _lc) in enumerate(reversed(live_boxes)):
                if lb == b:
                    del live_boxes[len(live_boxes) - li - 1]
                    break
        # TODO: later: record overlays at each time and retroactively wipe out ones that ended up scrolling/etc
        print "t", t, "nboxes:", len(live_boxes)
